In [7]:
from app.tool.action_planning import ActionPlanningTool
from app.llm import LLM
from app.schema import Message
import asyncio  
import nest_asyncio
nest_asyncio.apply()

user_msg = Message.user_message("Help me put the strawberries in the fridge")
system_msg = Message.system_message("You are an agent who can control a robotic arm and perform a series of actions using the arm")

llm = LLM()
planning_tool = ActionPlanningTool()

async def make_request():
    response = await llm.ask_tool(
        messages=[user_msg],
        system_msgs=[system_msg],
        tools=[planning_tool.to_param()],
        tool_choice="auto"
    )
    return response

response = asyncio.run(make_request())
print(response)

2025-03-27 13:01:33.365 | INFO     | app.llm:update_token_count:205 - Token usage: Input=720, Cumulative Input=1440


ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_16eacfee286c4f97b9df51', function=Function(arguments='{"plan_id": "1", "title": "Put strawberries in the fridge", "steps": [23, 21, 22, 24]}', name='action_planning'), type='function', index=0)])


In [8]:
import json
active_plan_id = "task001"

if response.tool_calls:
    for tool_call in response.tool_calls:
        # Parse the arguments
        args = tool_call.function.arguments
        if isinstance(args, str):
            try:
                args = json.loads(args)
            except json.JSONDecodeError:
                print(f"Failed to parse tool arguments: {args}")
                continue


        # Ensure plan_id is set correctly and execute the tool
        print("args:",args)
        # Execute the tool via ToolCollection instead of directly
        if tool_call.function.name == "action_planning":
            result = await planning_tool.execute(**args)
            print(result.output)

args: {'plan_id': '1', 'title': 'Put strawberries in the fridge', 'steps': [23, 21, 22, 24]}
Action plan created successfully with ID: 1

Plan: Put strawberries in the fridge (ID: 1)

Progress: 0/4 steps completed (0.0%)
Status: 0 completed, 0 in progress, 0 blocked, 4 not started

Steps:
0. [ ] Open the refrigerator door
1. [ ] Grab strawberries from the table
2. [ ] Put the strawberries in the refrigerator
3. [ ] Turn off the refrigerator



In [9]:
result = await planning_tool.execute(command="list")
print(result.output)


Available plans:
• 1 (active): Put strawberries in the fridge - 0/4 steps completed



In [10]:
result = await planning_tool.execute(
    command="create",
    plan_id="plan_001",
    title="My First Plan",
    steps=[23, 21, 22, 24]
)
print(result.output)


Action plan created successfully with ID: plan_001

Plan: My First Plan (ID: plan_001)

Progress: 0/4 steps completed (0.0%)
Status: 0 completed, 0 in progress, 0 blocked, 4 not started

Steps:
0. [ ] Open the refrigerator door
1. [ ] Grab strawberries from the table
2. [ ] Put the strawberries in the refrigerator
3. [ ] Turn off the refrigerator



In [11]:
result = await planning_tool.execute(command="list")
print(result.output)

Available plans:
• 1: Put strawberries in the fridge - 0/4 steps completed
• plan_001 (active): My First Plan - 0/4 steps completed



In [12]:
from app.tool.plan_validator import PlanValidator
from app.schema import Message
import asyncio  
import nest_asyncio
nest_asyncio.apply()
task = "Help me put the strawberries in the fridge"
user_msg = f"""
## Task
{task}
## Action plan sequence
{planning_tool.plan_to_prompt("plan_001")}
"""
user_msg = Message.user_message(user_msg)
system_msg = Message.system_message("You are an agent skilled in making independent judgments based on input requirements.")

plan_validator = PlanValidator()
async def make_request():
    response = await llm.ask_tool(
        messages=[user_msg],
        system_msgs=[system_msg],
        tools=[plan_validator.to_param()],
        tool_choice="auto"
    )
    return response

response = asyncio.run(make_request())
print(response)

2025-03-27 13:01:35.397 | INFO     | app.llm:update_token_count:205 - Token usage: Input=266, Cumulative Input=1706


ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_4a50ce6ad30342bfbf2b30', function=Function(arguments='{"status": "plan_generated_successfully"}', name='validate_plan'), type='function', index=0)])


In [13]:
if response.tool_calls:
    for tool_call in response.tool_calls:
        # Parse the arguments
        args = tool_call.function.arguments
        if isinstance(args, str):
            try:
                args = json.loads(args)
            except json.JSONDecodeError:
                print(f"Failed to parse tool arguments: {args}")
                continue
        print("status :",args.get("status"))

status : plan_generated_successfully


In [15]:
import asyncio  
import nest_asyncio
nest_asyncio.apply()


async def run_validator():
    result = await plan_validator.execute(task=task,
                                          plans=planning_tool.plan_to_prompt("plan_001"))
    return result

plan_pass = asyncio.run(run_validator())
if plan_pass:
    print("sucess")

2025-03-27 13:02:55.396 | INFO     | app.llm:update_token_count:205 - Token usage: Input=264, Cumulative Input=2234


sucess
